In [1]:

W = 64
H = 64

!pip install -q scikit-fuzzy
!cp -r /kaggle/input/inormalization/intensity-normalization-master/* ./
!python /kaggle/input/inormalization/intensity-normalization-master/setup.py install


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
import fnmatch

BASEPATH = '/kaggle/input/trends-assessment-prediction/'

dir_list = ['fMRI_train']
files = []
for directory in dir_list:
    for file in os.listdir(BASEPATH + directory + '/'):
        if fnmatch.fnmatch(file, '*.mat'):
            files.append((BASEPATH + directory + '/' + file, file[:-4]))
            
            
import nilearn.plotting as nlplt
import nilearn as nl
import nibabel as nib
import h5py
import matplotlib.pyplot as plt
import cv2
import time
iteration = 0


mask_filename = f"{BASEPATH}fMRI_mask.nii"
mask_niimg = nl.image.load_img(mask_filename)


def get_coord(image, index):
    x, y, z = nlplt.find_xyz_cut_coords(nl.image.index_img(image, index))
    coords = x, y, z
    return coords

files = sorted(files)

from intensity_normalization.normalize import fcm
# remamber to set slices correctly!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
for path, idx in files[:840]:
    
    start = time.time()
    
    all_maps = h5py.File(path, 'r')['SM_feature'][()]
    all_maps = np.moveaxis(all_maps, [0,1,2,3], [3,2,1,0])
    # load image into nifti file
    img_niimg = nl.image.new_img_like(mask_niimg, all_maps, affine=mask_niimg.affine, copy_header=True)
    wm_mask_any = fcm.find_tissue_mask(nl.image.mean_img(img_niimg), mask_niimg)
    subject_niimg = fcm.fcm_normalize(img_niimg, wm_mask_any, norm_value=1)
    # normalize data
    
    # caluculate cutting points
    cut_list_x = []
    cut_list_y = []
    cut_list_z = []

    for j in range(53):
        x, y, z = get_coord(subject_niimg, j)
        # compute voxel coords from mask coords using an inverse affine transformation
        x, y, z = nl.image.coord_transform(x, y, z,np.linalg.inv(mask_niimg.affine)) 
        cut_list_x.append(x)
        cut_list_y.append(y)
        cut_list_z.append(z)

    idx_3, idx_1, idx_2 = cut_list_x, cut_list_y, cut_list_z
    proj_1 = cv2.resize(np.array([np.stack(all_maps[i, int(idx_1[i]), :, :]) for i in range(53)]).transpose(1, 2, 0), (H, W))
    proj_2 = cv2.resize(np.array([np.stack(all_maps[i, :, int(idx_2[i]), :]) for i in range(53)]).transpose(1, 2, 0), (H, W))
    proj_3 = cv2.resize(np.array([np.stack(all_maps[i, :, :, int(idx_3[i])]) for i in range(53)]).transpose(1, 2, 0), (H, W))

    features = np.concatenate([proj_1, proj_2, proj_3], axis=2).transpose(2, 0, 1)
    print('{}nbytes'.format(features.nbytes))
    end = time.time()
    # save file
    np.save('{}'.format(idx), features)
    
    
    print(end - start)
    if iteration % 100 == 0:
        print(iteration)
    iteration += 1

You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.
running install
running bdist_egg
running egg_info
creating intensity_normalization.egg-info
writing intensity_normalization.egg-info/PKG-INFO
writing dependency_links to intensity_normalization.egg-info/dependency_links.txt
writing entry points to intensity_normalization.egg-info/entry_points.txt
writing top-level names to intensity_normalization.egg-info/top_level.txt
writing manifest file 'intensity_normalization.egg-info/SOURCES.txt'
reading manifest file 'intensity_normalization.egg-info/SOURCES.txt'
writing manifest file 'intensity_normalization.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib
creating build/lib/intensity_normalization
copying intensity_normalization/errors.py -> build/lib/intensity_normalization
copying intensity_normalization/__init__.py -> build/lib/intensity_n